# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

## 1. 取出今天所有的新聞

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from datetime import datetime, timedelta 
import pandas as pd
import numpy as np
import time
import random

chrome_options = Options()
#關閉瀏覽器跳出訊息
prefs = {
    'profile.default_content_setting_values' :
        {
        'notifications' : 2
         }
}
# options.add_experimental_option('prefs',prefs)
chrome_options.add_argument("--headless")            #不開啟實體瀏覽器背景執行
chrome_options.add_argument("--start-maximized")     #最大化視窗
chrome_options.add_argument("--incognito")           #開啟無痕模式

browser = webdriver.Chrome(executable_path='chromedriver', options = chrome_options)
browser.get("https://www.ettoday.net/news/news-list.htm")

for i in range(50):
    # browser.implicitly_wait(2)
    time.sleep(random.randint(2,5))
    # 移動到下方：    
    browser.execute_script("window.scrollTo(0, 80000);") # execute_script("填寫JS代碼")

print('finish')


finish


In [3]:
html_source = browser.page_source #網頁原始碼
soup = BeautifulSoup(html_source, "html5lib")
title_list = soup.find(class_="part_list_2").find_all('h3')

df = pd.DataFrame(data = np.zeros(shape=(len(title_list),4)), columns=['date', 'tag', 'title', 'address'])
id = 0
for t in title_list:
    df['date'][id] = t.find(class_='date').text
    df['tag'][id] = t.find(class_='tag').text
    df['title'][id] = t.find(target='_blank').text
    df['address'][id] = 'https://ettoday.net' + t.find('a')['href']
    id += 1

df['date'] = df['date'].apply(lambda x : datetime.strptime(x, "%Y/%m/%d %H:%M"))
print(df[df['date'] >= datetime.now().date()])
# print(df)
# browser.quit();

date   tag                                   title  \
0   2020-06-11 17:17:00    影劇             JK羅琳坦承被家暴、性侵！「要是晚生30年，我會變性」   
1   2020-06-11 17:17:00  寵物動物          汪姐車禍橫躺路邊！  浪浪弟「伴屍暖守護」…見人靠近伸手警戒   
2   2020-06-11 17:16:00    社會      「隨便賣，全收1萬2」臉書佯稱賣Switch主機　4嫌詐騙百萬遭聲押   
3   2020-06-11 17:15:00    大陸       女大生畢業典禮砍室友　2人被砍傷還要攻擊第3女...遭人奪刀阻止！   
4   2020-06-11 17:15:00  ET車雲  389匹節能油電休旅！BMW X5 xDrive45e開賣只要193.7萬起   
..                  ...   ...                                     ...   
578 2020-06-11 00:10:00    影劇             《陳情令》最美師姐披白紗！　宣璐搞定傲嬌BOSS出嫁了   
579 2020-06-11 00:10:00    時尚           進入微型世界拍「迷你婚紗照」！鑽戒當鞦韆、乘葉子渡河太浪漫   
580 2020-06-11 00:09:00    社會      左營情侶才為6500元房租大吵　「凌晨突大火」28歲男友床上燒成焦屍   
581 2020-06-11 00:05:00    政治         韓國瑜派人勸退凱道活動　主辦方回絕嗆「沒擔當」...真相曝光了   
582 2020-06-11 00:00:00    影劇     小甜甜熱戀眼鏡男…又爆財務危機！　求救謝忻「我比她還需要去咖啡廳打工」   

                                           address  
0    https://ettoday.net/news/20200611/1735658.htm  
1    https://ettoday.net/news/202006

## 2. 取出現在時間兩小時內的新聞

In [4]:

now = datetime.now()
two_hour = now - timedelta(hours = 2)

print(df[df['date'] >= two_hour])



date   tag                                   title  \
0   2020-06-11 17:17:00    影劇             JK羅琳坦承被家暴、性侵！「要是晚生30年，我會變性」   
1   2020-06-11 17:17:00  寵物動物          汪姐車禍橫躺路邊！  浪浪弟「伴屍暖守護」…見人靠近伸手警戒   
2   2020-06-11 17:16:00    社會      「隨便賣，全收1萬2」臉書佯稱賣Switch主機　4嫌詐騙百萬遭聲押   
3   2020-06-11 17:15:00    大陸       女大生畢業典禮砍室友　2人被砍傷還要攻擊第3女...遭人奪刀阻止！   
4   2020-06-11 17:15:00  ET車雲  389匹節能油電休旅！BMW X5 xDrive45e開賣只要193.7萬起   
..                  ...   ...                                     ...   
124 2020-06-11 15:30:00  寵物動物               聽見女娃哭了！柴犬哥哥衝在主人前　網泛淚：可以依靠   
125 2020-06-11 15:29:00    地方               新北勞大20週年回顧熱烈開展　首屆結業生齊聚回娘家   
126 2020-06-11 15:29:00    論壇                     王志鵬／對釣魚台應強硬　台灣勿軟弱無力   
127 2020-06-11 15:28:00    政治          韓國瑜最終上班日！　鄉民憶去年情景...預測下一步：很難不選   
128 2020-06-11 15:27:00    影劇          生命誕生前爸爸被打、阿公被抓！《當愛再來的時候》給大人的提醒   

                                           address  
0    https://ettoday.net/news/20200611/1735658.htm  
1    https://ettoday.net/news/202006

## 3. 根據範例，取出三天前下午三點到五點的新聞

In [33]:
now = datetime.now().date()
three_days = now - timedelta(days = 3)
three_days_str = three_days.strftime("%Y-%m-%d")

# browser = webdriver.Chrome(executable_path='chromedriver')
browser = webdriver.Chrome(executable_path='chromedriver', options = chrome_options)
browser.get("https://www.ettoday.net/news/news-list-{}-0.htm".format(three_days_str))

for i in range(50):
    # browser.implicitly_wait(2)
    time.sleep(random.randint(2,5))
    # 移動到下方：    
    browser.execute_script("window.scrollTo(0, 80000);") # execute_script("填寫JS代碼")

print('finish')

finish


In [34]:
html_source = browser.page_source #網頁原始碼
soup = BeautifulSoup(html_source, "html5lib")
title_list = soup.find(class_="part_list_2").find_all('h3')

df = pd.DataFrame(data = np.zeros(shape=(len(title_list),4)), columns=['date', 'tag', 'title', 'address'])
id = 0
for t in title_list:

    df['date'][id] = t.find(class_='date').text
    df['tag'][id] = t.find(class_='tag').text
    df['title'][id] = t.find(target='_blank').text
    df['address'][id] = 'https://ettoday.net' + t.find('a')['href']
    id += 1


a = datetime(three_days.year, three_days.month, three_days.day)
print(a)
a = a + timedelta(hours = 15)
b = a + timedelta(hours = 2)
df['date'] = df['date'].apply(lambda x : datetime.strptime(x, "%Y/%m/%d %H:%M"))
print(df[(df['date'] >= a) & (df['date'] <= b)])
# print(df)

2020-06-09 00:00:00
                   date tag                                title  \
243 2020-06-09 17:00:00  旅遊        肉肉控快衝！板橋平價厚切牛排超大塊　濃湯、沙拉吧免費嗑到飽   
244 2020-06-09 16:59:00  財經                 金管會提早取消限空令！　台股3大解封措施   
245 2020-06-09 16:59:00  地方              山城南投閱讀無障礙　開啟偏鄉兒童廣大的閱讀世界   
246 2020-06-09 16:59:00  財經            疫情加速新消費趨勢轉型　法人看好宅經濟概念股再創高   
247 2020-06-09 16:59:00  國際  租屋處驚見「超詭異巨型膠囊」！網神解「昭和奢侈品」...竟獲1萬次轉推   
..                  ...  ..                                  ...   
378 2020-06-09 15:03:00  時尚         7個簡單小動作幫你趕走壞情緒　整理房間、假裝笑臉都有用！   
379 2020-06-09 15:02:00  社會         還韓國瑜公道！杏仁哥籲號召6/13上凱道　北市警局核准了   
380 2020-06-09 15:01:00  體育         疫情衝擊無賽事影響生計　前大馬羽球國家隊男雙暫時改賣榴槤   
381 2020-06-09 15:00:00  影劇           全昭旻遭連署退出、詛咒全家　《RM》把觀眾留言板關了   
382 2020-06-09 15:00:00  旅遊      泡「彈珠汽水」消暑！南方澳一日遊攻略　神秘海岸內行人帶路才能到   

                                           address  
243  https://ettoday.net/news/20200609/1730557.htm  
244  https://ettoday.net/news/20200609/1733864.htm  
245 